In [38]:
import pandas as pd
import mysql.connector as mysql

# Establishing Connection

In [7]:
db = mysql.connect(host="localhost",port="3306",user="root",password="admin1234#",database="sales")

In [13]:
customers = pd.read_sql_query("select * from customers",db)

C:\Users\DELL PC\AppData\Local\Temp\ipykernel_10332\1805277540.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customers = pd.read_sql_query("select * from customers",db)


In [14]:
customers.shape

(38, 3)

In [16]:
date = pd.read_sql_query("select * from date",db)

C:\Users\DELL PC\AppData\Local\Temp\ipykernel_10332\3097153016.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  date = pd.read_sql_query("select * from date",db)


In [17]:
date.shape

(1126, 5)

In [19]:
markets = pd.read_sql_query("select * from markets",db)

C:\Users\DELL PC\AppData\Local\Temp\ipykernel_10332\1889616315.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  markets = pd.read_sql_query("select * from markets",db)


In [20]:
markets.shape

(17, 3)

In [21]:
products = pd.read_sql_query("select * from products",db)

C:\Users\DELL PC\AppData\Local\Temp\ipykernel_10332\3538033511.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  products = pd.read_sql_query("select * from products",db)


In [22]:
products.shape

(279, 2)

In [23]:
transactions = pd.read_sql_query("select * from transactions",db)

C:\Users\DELL PC\AppData\Local\Temp\ipykernel_10332\1581133205.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  transactions = pd.read_sql_query("select * from transactions",db)


In [25]:
transactions.shape

(150283, 7)

# Writing excel files

In [45]:
transactions.to_csv("transactions.csv",index=False)
markets.to_csv("markets.csv",index=False)
date.to_csv("date.csv",index=False)
products.to_csv("products.csv",index=False)
customers.to_csv("customers.csv",index=False)

# Cleaning excel files

### 1) Transaction column

In [108]:
transactions.head(3)

,product_code,customer_code,market_code,order_date,sales_qty,sales_amount,currency
0,Prod001,Cus001,Mark001,2017-10-10,100,41241.0,INR
1,Prod001,Cus002,Mark002,2018-05-08,3,-1.0,INR
2,Prod002,Cus003,Mark003,2018-04-06,1,875.0,INR


In [109]:
df = transactions.copy()

In [110]:
df.head(2)

,product_code,customer_code,market_code,order_date,sales_qty,sales_amount,currency
0,Prod001,Cus001,Mark001,2017-10-10,100,41241.0,INR
1,Prod001,Cus002,Mark002,2018-05-08,3,-1.0,INR


In [111]:
df.loc[(df["sales_amount"]==-1) | (df["sales_amount"]==0) ,:].count()

product_code     1611
customer_code    1611
market_code      1611
order_date       1611
sales_qty        1611
sales_amount     1611
currency         1611
dtype: int64

In [127]:
condition1 = (df["sales_amount"]==0)
condition2 = (df["sales_amount"]==-1)

index_to_delete = df[condition1 | condition2].index
df = df.drop(index_to_delete)

In [122]:
df["currency"].unique()

array(['INR'], dtype=object)

In [123]:
df["currency"] = df["currency"].str.strip()

In [124]:
df.loc[df["currency"]=="USD",:]

,product_code,customer_code,market_code,order_date,sales_qty,sales_amount,currency


## Conditionally added new column

In [116]:
import numpy as np
df["sales_amount"] = np.where(df["currency"]=="USD",(df["sales_amount"]*75),df["sales_amount"])

In [117]:
df["currency"] = np.where(df["currency"]=="USD","INR","INR")

In [118]:
df

,product_code,customer_code,market_code,order_date,sales_qty,sales_amount,currency
0,Prod001,Cus001,Mark001,2017-10-10,100,41241.0,INR
2,Prod002,Cus003,Mark003,2018-04-06,1,875.0,INR
3,Prod002,Cus003,Mark003,2018-04-11,1,583.0,INR
4,Prod002,Cus004,Mark003,2018-06-18,6,7176.0,INR
5,Prod003,Cus005,Mark004,2017-11-20,59,37500.0,INR
...,...,...,...,...,...,...,...
150278,Prod339,Cus005,Mark004,2019-04-18,1,394.0,INR
150279,Prod339,Cus020,Mark004,2019-04-23,1,667.0,INR
150280,Prod339,Cus007,Mark004,2019-04-23,1,625.0,INR
150281,Prod339,Cus006,Mark004,2019-04-24,7,8625.0,INR


In [119]:
df.to_csv("transactions.csv",index=False)